In [1]:
import pandas as pd

In [2]:
import pandas as pd


df = pd.read_csv("/Users/sebastianvinther/Desktop/Sportsmonks/fixtures_full.csv", low_memory=False)
print("📄 Columns in your fixtures file:")
print(df.columns.tolist())

📄 Columns in your fixtures file:
['id', 'sport_id', 'league_id', 'season_id', 'stage_id', 'group_id', 'aggregate_id', 'round_id', 'state_id', 'venue_id', 'name', 'starting_at', 'result_info', 'leg', 'details', 'length', 'placeholder', 'has_odds', 'has_premium_odds', 'starting_at_timestamp']


In [ ]:
import pandas as pd
import json

# === CONFIG ===
DETAILS_FILE = "/Users/sebastianvinther/Desktop/Sportsmonks/fixture_details.csv"

# === LOAD A FEW FIXTURES TO INSPECT ===
fixtures = []
with open(DETAILS_FILE, "r") as f:
    for i, line in enumerate(f):
        try:
            fixture = json.loads(line)
            fixtures.append(fixture)
        except:
            continue
        if i >= 20:  # Check first 20 only
            break

# === CHECK PARTICIPANT STRUCTURE ===
found_meta = 0
missing_meta = 0

for fixture in fixtures:
    participants = fixture.get("participants", [])
    for p in participants:
        meta = p.get("meta", {})
        if meta.get("location") in {"home", "away"}:
            found_meta += 1
        else:
            missing_meta += 1

# === RESULT ===
if found_meta > 0:
    print(f"✅ Found meta.location for {found_meta} participants.")
else:
    print("❌ No meta.location found in participants — you'll need to re-fetch with includes.")

if missing_meta > 0:
    print(f"⚠️ {missing_meta} participants were missing meta.location.")


In [1]:
import pandas as pd
import requests
import time
import os
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
from datetime import datetime

# === CONFIG ===
API_TOKEN = "oYeoAVFUTQpu7MfoFqbvyiYfgRRkuBWW0p2atkZnySe4X3xrHkjgGhOvI0pd"
FIXTURE_FILE = "/Users/sebastianvinther/Desktop/Sportsmonks/fixtures_full.csv"
OUTPUT_FILE = "/Users/sebastianvinther/Desktop/Sportsmonks/fixture_details.csv"
SAVE_EVERY = 100
NUM_THREADS = 2
MAX_CALLS_PER_HOUR = 53000
DELAY = 3600 / MAX_CALLS_PER_HOUR  # ~0.068 seconds between calls

# === LOAD FIXTURE IDS ===
fixture_ids = pd.read_csv(FIXTURE_FILE)['id'].dropna().astype(int).tolist()

# === LOAD PROGRESS ===
if os.path.exists(OUTPUT_FILE):
    existing = pd.read_csv(OUTPUT_FILE)
    done_ids = existing['id'].unique().tolist()
    remaining_ids = [fid for fid in fixture_ids if fid not in done_ids]
    print(f"🔁 Resuming: {len(done_ids)} done, {len(remaining_ids)} remaining.")
else:
    existing = None
    remaining_ids = fixture_ids
    print(f"🆕 Starting fresh with {len(remaining_ids)} fixtures.")

# === FLATTEN FUNCTION ===
def flatten_fixture(data):
    flat = {
        'id': data.get('id'),
        'name': data.get('name'),
        'starting_at': data.get('starting_at'),
        'league_id': data.get('league', {}).get('id'),
        'league_name': data.get('league', {}).get('name'),
        'venue_id': data.get('venue', {}).get('id'),
        'venue_name': data.get('venue', {}).get('name'),
        'state_id': data.get('state', {}).get('id'),
        'state_name': data.get('state', {}).get('name'),
        'weather_temp': data.get('weatherReport', {}).get('temperature_celcius')
    }

    for p in data.get('participants', []):
        loc = p.get('meta', {}).get('location')
        if loc in ['home', 'away']:
            flat[f'{loc}_team_id'] = p.get('id')
            flat[f'{loc}_team_name'] = p.get('name')

    return flat

# === API CALL ===
def fetch_fixture(fid):
    url = f"https://api.sportmonks.com/v3/football/fixtures/{fid}"
    params = {
        "api_token": API_TOKEN,
        "include": "participants;league;venue;state;weatherReport"
    }
    try:
        response = requests.get(url, params=params, timeout=10)
        time.sleep(DELAY)  # throttle
        if response.status_code == 200:
            data = response.json().get("data")
            return flatten_fixture(data)
        else:
            print(f"[{fid}] Status {response.status_code}")
    except Exception as e:
        print(f"[{fid}] Error: {e}")
    return None

# === FETCH IN PARALLEL WITH PERIODIC SAVING ===
buffer = []
with ThreadPoolExecutor(max_workers=NUM_THREADS) as executor:
    futures = {executor.submit(fetch_fixture, fid): fid for fid in remaining_ids}
    for i, future in enumerate(tqdm(as_completed(futures), total=len(futures), desc="Fetching fixtures")):
        result = future.result()
        if result:
            buffer.append(result)

        if len(buffer) >= SAVE_EVERY:
            df = pd.DataFrame(buffer)
            df.to_csv(OUTPUT_FILE, mode='a', header=not os.path.exists(OUTPUT_FILE), index=False)
            print(f"💾 Saved {len(buffer)} at {datetime.now().strftime('%H:%M:%S')}")
            buffer = []

# Final save
if buffer:
    pd.DataFrame(buffer).to_csv(OUTPUT_FILE, mode='a', header=not os.path.exists(OUTPUT_FILE), index=False)
    print(f"✅ Final save: {len(buffer)} fixtures.")

print("🎯 Done! All fixture details saved.")

/var/folders/nl/dl4x3l3x1dd8twj43d1ddkx80000gn/T/ipykernel_39917/4105866195.py:19: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  fixture_ids = pd.read_csv(FIXTURE_FILE)['id'].dropna().astype(int).tolist()


🔁 Resuming: 11500 done, 144041 remaining.


Fetching fixtures:   0%|          | 102/144041 [00:10<4:13:30,  9.46it/s]

💾 Saved 100 at 19:48:24


Fetching fixtures:   0%|          | 200/144041 [00:21<4:12:49,  9.48it/s]

...

💾 Saved 100 at 00:15:46


Fetching fixtures: 100%|█████████▉| 143949/144041 [4:27:43<00:09,  9.42it/s]

💾 Saved 100 at 00:15:56


Fetching fixtures: 100%|██████████| 144041/144041 [4:27:52<00:00,  8.96it/s]

✅ Final save: 92 fixtures.
🎯 Done! All fixture details saved.


In [ ]:
# === CONFIG ===
FIXTURE_FILE = "/Users/sebastianvinther/Desktop/Sportsmonks/fixtures_full.csv"
TEAM_SEASON_FILE = "/Users/sebastianvinther/Desktop/Sportsmonks/team_season_combinations.csv"

# === LOAD FIXTURE DATA ===
fixtures_df = pd.read_csv(FIXTURE_FILE, low_memory=False)

# === EXTRACT UNIQUE (season_id, team_id) COMBINATIONS ===
home_teams = fixtures_df[['season_id', 'home_team_id']].rename(columns={'home_team_id': 'team_id'})
away_teams = fixtures_df[['season_id', 'away_team_id']].rename(columns={'away_team_id': 'team_id'})
all_teams = pd.concat([home_teams, away_teams]).dropna().drop_duplicates()

# === SAVE TO CSV ===
all_teams = all_teams.astype({'season_id': int, 'team_id': int})
all_teams.to_csv(TEAM_SEASON_FILE, index=False)

print(f"✅ Extracted and saved {len(all_teams)} (season_id, team_id) pairs to {TEAM_SEASON_FILE}")
